<a href="https://colab.research.google.com/github/Highpointer/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-17 18:51:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.95MB/s    in 0.2s    

2022-09-17 18:51:54 (4.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [65]:
from pyspark import SparkFiles
url = "https://highpointer.s3.amazonaws.com/amazon_reviews_us_Outdoors_v1_00.tsv" # replace this url
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show(50, truncate=False)

+-----------+-----------+--------------+----------+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Create DataFrames to match tables

In [36]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [77]:
# Create the customers_table DataFrame
#customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "Customer Count").withColumnRenamed("customer_id", "Customer ID")
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43679767|             1|
|   32024654|             1|
|   52913169|             1|
|   24297214|             1|
|   26096454|             1|
|   38247118|             1|
|   44009906|             1|
|    1753876|             1|
|   26195644|             1|
|   23042837|             1|
|   11487525|             2|
|   28258386|             1|
|   42560427|             2|
|   24540309|             1|
|    1967239|             1|
|   14217455|             1|
|   20289484|            26|
|   47103434|             1|
|   28638887|             1|
|     740678|             1|
+-----------+--------------+
only showing top 20 rows



In [69]:
# Create the products_table DataFrame and drop duplicates. 
#products_df = df.select(["product_id", "product_title"]).drop_duplicates().withColumnRenamed("product_id", "Product ID").withColumnRenamed("product_title", "Product Title")
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(20, truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                 |
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B00IFHFJXI|Ivation Portable Outdoor Shower, Battery Powered - Compact Handheld Rechargeable Camping Showerhead - Pumps Water from Bucket Into Steady, Gentle Shower Stream                               |
|B00WG0J0D0|JanSport Superbreak Backpack                                                                                                                                                

In [70]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
#review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).withColumnRenamed("review_id", "Review ID").withColumnRenamed("customer_id", "Customer ID").withColumnRenamed("product_id", "Product ID").withColumnRenamed("product_parent", "Product Parent").withColumnRenamed("review_date", "Review Date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(20, truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R35T75OLUGHL5C|18446823   |B000NV6H94|110804376     |2015-08-31 |
|R2BV735O46BN33|13724367   |B000IN0W3Y|624096774     |2015-08-31 |
|R2NBEUGPQQGXP1|51001958   |B008RBJXFM|278970944     |2015-08-31 |
|R17LLAOJ8ITK0S|32866903   |B00FK8WUQY|312877650     |2015-08-31 |
|R39PEQBT5ISEF4|30907790   |B00EZA3VW0|305567912     |2015-08-31 |
|R3GNM3SU9VHJFT|20232229   |B006JA8WEG|842306035     |2015-08-31 |
|R2Y81OP0EK467 |17698862   |B002PWFSEO|451480122     |2015-08-31 |
|R2LFGSI6HAYH5F|38486114   |B002DZGKHW|124386306     |2015-08-31 |
|R297G6ED1IQO7W|26319572   |B00ABA08F6|991442421     |2015-08-31 |
|RE27RFC6101N6 |27152337   |B003Z8WIHC|886483892     |2015-08-31 |
|R3BPDME6E94W8Z|12516845   |B007CP6UK0|150224054     |2015-08-31 |
|R2P08O1RILUOX3|3225242    |B003V3U9JK|343847969     |2015-08-

In [72]:
# Create the vine_table. DataFrameer
#vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).withColumnRenamed("review_id", "Review ID").withColumnRenamed("star_rating", "Star Rating").withColumnRenamed("helpful_votes", "Helpful Votes").withColumnRenamed("total_votes", "Total Votes").withColumnRenamed("vine", "Vine").withColumnRenamed("verified_purchase", "Verified Purchase")
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(20, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|4          |0            |0          |N   |Y                |
|R2BV735O46BN33|5          |0            |0          |N   |Y                |
|R2NBEUGPQQGXP1|4          |0            |0          |N   |Y                |
|R17LLAOJ8ITK0S|3          |1            |1          |N   |Y                |
|R39PEQBT5ISEF4|1          |0            |0          |N   |Y                |
|R3GNM3SU9VHJFT|4          |1            |1          |N   |Y                |
|R2Y81OP0EK467 |5          |0            |0          |N   |Y                |
|R2LFGSI6HAYH5F|5          |1            |1          |N   |Y                |
|R297G6ED1IQO7W|5          |1            |1          |N   |Y                |
|RE27RFC6101N6 |5          |0            |0          |N   |Y    

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [63]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://deliverable1.cnnbrujhuvz0.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "REDACTED", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [78]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [79]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)